In [7]:
import torch
from torch import nn
import torchvision
from PIL import Image

device = torch.device('mps')

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=2)

# for batch, labels in train_loader:
#     print(batch)
#     print(batch.shape)
#     print(labels.shape)
#     break

In [9]:
from vision_transformer import VisionTransformer

device = torch.device('mps')

model = VisionTransformer().to(device)

x = torch.randn(size=(2, 3, 32, 32), device=device)

out = model(x)

print(out)
print(out.shape)

tensor([[[ 1.8869,  0.8311, -1.7552,  ...,  1.4471, -0.9362, -1.1161],
         [ 1.0313, -0.3949, -0.4613,  ...,  1.7935,  0.9795,  0.7702],
         [-0.9208, -1.7588, -1.2951,  ...,  0.8002,  1.8259,  0.2337],
         ...,
         [-0.1181,  0.9076, -0.6228,  ..., -0.3087, -0.7735,  0.0075],
         [ 1.3424,  2.0564,  0.4536,  ..., -0.6976,  0.1727,  0.9069],
         [ 2.4312, -0.9827, -0.6746,  ...,  2.1041,  0.1809,  0.2285]],

        [[ 1.8869,  0.8311, -1.7552,  ...,  1.4471, -0.9362, -1.1161],
         [ 1.7213,  0.0170,  0.2140,  ...,  1.0390,  0.3611,  0.3375],
         [-1.4594, -2.5140, -1.8907,  ...,  1.1139,  2.3375,  0.5051],
         ...,
         [-0.1397, -0.1788, -1.3071,  ..., -0.0777, -0.4168,  0.4978],
         [ 0.6216,  1.0749,  1.0680,  ..., -0.1775,  0.5356,  1.4408],
         [ 0.5734, -1.8890, -1.0988,  ...,  1.4712, -0.2528, -1.0735]]],
       device='mps:0', grad_fn=<AddBackward0>)
torch.Size([2, 65, 512])
